# Boxing Simulation

## Patrick Maloney
### DATA604 Final Project

## Introduction

Boxing is one of the world's oldest professional sports, but it has been one of the slowest to be inoavted by analytics and data science applications that now proliferate other professional sports. In this project, I will attempt to build a simulation of the biggest match in history, Floyd Mayweather Jr. vs. Manny Pacquiao.  The match was one of the biggest financial events in the history of sports, netting each fighter nine-figure paydays for an evening's work, grossing over \$600 million overall, and leading to a then-record of over \\$50 million dollars being wagered on the fights outcome.  I will attempt to use punch statistics from both fighters to simulate the fight with Python.


### Import libraries

In [1]:
import numpy
import pandas
import random
from modsim import *
%matplotlib inline
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
# set the random number generator
np.random.seed(7)

### Build State objects for each fighter

I will be using the modsim library for its convenience in building simulations. I build a state object for each fighter, which will  store each of there metrics from the simulaiton.

In [2]:
mayweather = State(thrown = 0, landed = 0, kd = 0, score = 0)
pacquiao = State(thrown = 0, landed = 0, kd = 0, score = 0)

,values
thrown,0
landed,0
kd,0
score,0


### Time-step function
Now we need to create a time-step function that will simulate an instance in a round, so that we can later loop over to simulate an entire round. In this step function, we will use punch statistics to determine the probabilities that a punch will be thrown during a given moment, and whether that punch will land. 

In [3]:
def step(system):
    """This function takes in a system argument 
    containing punch probabilities and increments
    the state objects for each fighter"""
    if flip(system.m_throw): #probability Mayweather throws a punch
        mayweather.thrown += 1 # increment Mayweather "thrown" stats
        print('may throw')
        if flip(system.m_land): #probability the thrown punch lands on Pacquiao
            mayweather.landed += 1 # increment Mayweather "landed" stats
            print('may land')

In [4]:
# We define a system of parameters that correspond with the probability that each fighter
# will throw and/or land a punch during any given second of a round. these are placeholder values to get started.
system = System(m_throw = 0.21, m_land = 0.48, p_throw = 0.28, p_land = 0.3 )

,values
m_throw,0.21
m_land,0.48
p_throw,0.28
p_land,0.30


In [5]:
step(system)

may throw


In [6]:
mayweather

,values
thrown,1
landed,0
kd,0
score,0


We see the function works as intended for a single fighter. Now let's add the second fighter to the mix, since boxing takes two to tango.

In [7]:
def step(system):
    """This function takes in a system argument 
    containing punch probabilities and increments
    the state objects for each fighter"""
    #Mayweather
    if flip(system.m_throw): #probability Mayweather throws a punch
        mayweather.thrown += 1 # increment Mayweather "thrown" stats
        print('may throw')
        if flip(system.m_land): #probability the thrown punch lands on Pacquiao
            mayweather.landed += 1 # increment Mayweather "landed" stats
            print('may land')
    #Pacquiao
    if flip(system.p_throw): #probability Pacquiao throws a punch
        pacquiao.thrown += 1 # increment Pacquiao "thrown" stats
        print('pac throw')
        if flip(system.p_land): #probability the thrown punch lands on Mayweather
            pacquiao.landed += 1 # increment Pacquiao "landed" stats
            print('pac land')

In [8]:
step(system)

In [9]:
mayweather

,values
thrown,1
landed,0
kd,0
score,0


In [10]:
pacquiao

,values
thrown,0
landed,0
kd,0
score,0


It worked as intended after I ran the cell multiple times. The next thing I will need is a function that will initialize the state objects back to zero to make testing these functions easier.

In [11]:
def reset_stats(state):
    "Takes state object as argument and resets punch counts"
    state.thrown = 0
    state.landed = 0
    state.kd = 0
    state.score = 0

reset_stats(mayweather)
mayweather

,values
thrown,0
landed,0
kd,0
score,0


In [12]:
reset_stats(pacquiao)
pacquiao

,values
thrown,0
landed,0
kd,0
score,0


### Simulating a round
Now lets run the time-step function in a loop 180 times to simulate a 3-minute round. At the end of the round, we will want to compare the fighters landed punches and award more points to the person who landed more punches. Boxing uses a scoring system that awards 10 points to the winner and 9 points to the loser (minus a point for a knockdown, but we'll deal with that later.)  I will also remove the print statements from the step function.

In [13]:
def step(system):
    """This function takes in a system argument 
    containing punch probabilities and increments
    the state objects for each fighter"""
    #Mayweather
    if flip(system.m_throw): #probability Mayweather throws a punch
        mayweather.thrown += 1 # increment Mayweather "thrown" stats
        if flip(system.m_land): #probability the thrown punch lands on Pacquiao
            mayweather.landed += 1 # increment Mayweather "landed" stats
  
    #Pacquiao
    if flip(system.p_throw): #probability Pacquiao throws a punch
        pacquiao.thrown += 1 # increment Pacquiao "thrown" stats
        if flip(system.p_land): #probability the thrown punch lands on Mayweather
            pacquiao.landed += 1 # increment Pacquiao "landed" stats

            
def sim_round(system):
    for i in range(180): # run step function for each second in 3 minute round
        step(system)
    if mayweather.landed > pacquiao.landed:
        mayweather.score += 10
        pacquiao.score += 9
    elif pacquiao.landed > mayweather.landed:
        mayweather.score += 9
        pacquiao.score += 10
    else:
        mayweather.score += 10
        pacquiao.score += 10
        

sim_round(system)
mayweather

,values
thrown,42
landed,17
kd,0
score,10


In [14]:
pacquiao

,values
thrown,52
landed,15
kd,0
score,9


The round_sim function appears to work as intended. However, for it to work in a loop to simulate an entire fight, we will need  the counts to reset at the beginning of each round after the points have been awarded, but still increment the totals for each fighter. To do this, I will create a new state object within the system object that will store the values for that round and increment the totals for each fighter. I will also include a new function to reset the values at the beginning of each round.

In [65]:
init = State(m_thrown = 0,
             m_landed = 0,
             m_kd = 0,
             p_thrown = 0,
             p_landed = 0,
             p_kd = 0)

system = System(init = init,
                m_throw = 0.21,
                m_land = 0.48,
                p_throw = 0.28,
                p_land = 0.30)

def reset_round_stats():
    system.init = State(m_thrown = 0,
             m_landed = 0,
             m_kd = 0,
             p_thrown = 0,
             p_landed = 0,
             p_kd = 0)

def step(system):
    """This function takes in a system argument 
    containing punch probabilities and increments
    the state objects for each fighter"""
    #Mayweather
    if flip(system.m_throw): #probability Mayweather throws a punch
        system.init.m_thrown += 1 # increment Mayweather round "thrown" stats
        if flip(system.m_land): #probability the thrown punch lands on Pacquiao
            system.init.m_landed += 1 # increment Mayweather "landed" stats for this round
  
    #Pacquiao
    if flip(system.p_throw): #probability Pacquiao throws a punch
        system.init.p_thrown += 1 # increment Pacquiao "thrown" stats
        if flip(system.p_land): #probability the thrown punch lands on Mayweather
            system.init.p_landed += 1 # increment Pacquiao "landed" stats

            
def sim_round(system):
    """Takes in a system object that includes punch counters for the round.
    Passes the system argument through the step function in a loop.
    increments state object parameters based on loop results."""
    
    reset_round_stats() # clear punch stats of previous round
    for i in range(180): # run step function for each second in 3 minute round
        step(system)
    if system.init.m_landed > system.init.p_landed:
        mayweather.score += 10
        pacquiao.score += 9
    elif system.init.p_landed > system.init.m_landed:
        mayweather.score += 9
        pacquiao.score += 10
    else:
        mayweather.score += 10
        pacquiao.score += 10
    # transfer punch stats for this round to cumulative totals
    mayweather.thrown += system.init.m_thrown 
    mayweather.landed += system.init.m_landed
    mayweather.kd += system.init.m_kd
    pacquiao.thrown += system.init.p_thrown
    pacquiao.landed += system.init.p_landed
    pacquiao.kd += system.init.p_kd

In [70]:
reset_stats(mayweather)
reset_stats(pacquiao)
sim_round(system)
system.init

,values
m_thrown,35
m_landed,21
m_kd,0
p_thrown,44
p_landed,17
p_kd,0


In [71]:
mayweather

,values
thrown,35
landed,21
kd,0
score,10


In [72]:
pacquiao

,values
thrown,44
landed,17
kd,0
score,9
